In [ ]:
# 크롤링에 필요한 패키지 설치
!python -m pip install selenium
!python -m pip install webdriver-manager

  Using cached selenium-4.31.0-py3-none-any.whl.metadata (7.5 kB)
  Using cached trio-0.29.0-py3-none-any.whl.metadata (8.5 kB)
  Using cached trio_websocket-0.12.2-py3-none-any.whl.metadata (5.1 kB)
  Using cached websocket_client-1.8.0-py3-none-any.whl.metadata (8.0 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl.metadata (10 kB)
  Using cached outcome-1.3.0.post0-py2.py3-none-any.whl.metadata (2.6 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached PySocks-1.7.1-py3-none-any.whl.metadata (13 kB)
  Using cached pycparser-2.22-py3-none-any.whl.metadata (943 bytes)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
Using cached selenium-4.31.0-py3-none-any.whl (9.4 MB)
Using cached trio-0.29.0-py3-none-any.whl (492 kB)
Using cached trio_websocket-0.12.2-py3-none-any.whl (21 kB)
Using cached websocket_client

In [1]:
# 크롤링에 필요한 패키지 불러오기.
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

In [2]:
# 1. 크롬 브라우저 자동 설정
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

[코드 설명]
- `webdriver.ChromeOptions()`: 크롬 드라이버 실행 옵션 설정 객체. (창 크기 조절 등)
- `.add_argument('--start-maximized')`: options 객체에 실행 시 브라우저를 최대화 상태로 시작하라는 인자 추가.
    - 다른 인자)
        - `--headless` : 브라우저 창 없이 백그라운드로 실행
        - `--disable-popup-blocking` : 팝업 차단 해제
        - `--incognito` : 시크릿 모드로 실행
        - `--window-size=1920,1080`: 브라우저 창 크기 지정
- `webdriver.Chrome(...)`: 실제로 **크롬 브라우저 인스턴스를 실행**하는 메서드.
    - `service=Service(...)`: ChromeDriver를 직접 설치하지 않아도 자동으로 설치하고 실행하게 해주는 도우미.
    - `options=options`: 위에서 만든 options 객체를 적용해 브라우저 실행 방식을 조절.
- `ChromeDriverManager().install()`: 자동으로 최신 드라이버 설치.

In [3]:
# 2. 수집할 URL로 이동
url = "https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=100000100020006"
driver.get(url)
time.sleep(5)   # 페이지 로딩 대기

[코드 설명]
- driver.get(url): 해당 URL로 브라우저 이동.
- 동적 로딩이 많아 time.sleep()으로 로딩을 기다리는 방식 필요, 추후 WebDriverWait으로 개선하면 더 안정적.

In [4]:
# 3. 제품 페이지로 이동 (예시로 첫 번째 제품 클릭)
product_links = driver.find_elements(By.CSS_SELECTOR, '.prd_info .tx_name')
product_links[0].click()
time.sleep(5)

[코드 설명]
- `.find_elements()`: 여러 요소를 찾아 리스트로 반환.
    - 위의 코드의 목적은 **상품 이름을 담고 있는 a 태그(=제품 상세 링크)**를 모두 찾는 것.
    - `'.prd_info .tx_name'` 은 HTML 문서 내 < div > 태그의 클래스명. HTML안의 클래스를 CSS에서 선택할 때는 반드시 '.'를 붙여야 함.
    - 즉, 위의 코드는 클래스명이 .prd_info
- CSS 선택자 문법
    - `.className`: 클래스 선택
    - `#idName`: ID 선택
    - `tagname`: 태그 선택
    - `div.className`: 특정 태그+클래스 (ex: div.prd_info = < div class="prd_info" >)
    - `.class1 .class2`: 클래스 중첩 선택. (class1 안에 있는 class2의 요소)

In [5]:
# 4. 리뷰 탭으로 이동
driver.switch_to.window(driver.window_handles[-1])
review_tab = driver.find_element(By.CSS_SELECTOR, 'a.goods_reputation')
review_tab.click()
time.sleep(5)

[코드 설명]
- `href="javascript:;`: 링크가 실제 페이지 이동이 아니라 JavaScript로 이벤트 트리거함. 따라서 Selenium으로 `.click()` 해야만 리뷰 내용이 로딩됨.

In [13]:
# 5. 리뷰 수집

# 리뷰 탭 클릭
from selenium.common.exceptions import TimeoutException   # 예외처리(except)를 위해 import.
try:
    # 리뷰 탭이 클릭 가능한 상태가 될 때 까지 대기기
    review_tab = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, 'a.goods_reputation'))
    )
    driver.execute_script("arguments[0].click();", review_tab)   # JS 클릭으로 안정성 ↑
    time.sleep(2)   # JS 로딩 시간 대기
    
    # 리뷰 목록이 나타날 때 까지 대기
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'ul#gdasList li'))
    )
    print("✅ 리뷰 탭 클릭 및 로딩 완료")
except TimeoutException:
    print("❌ 리뷰 탭 또는 리뷰 목록 로딩 실패")
    driver.quit()

# 리뷰 수집 시작
data = []
for i in range(60):   # 리뷰 페이지 수 만큼 반복
    
    reviews = driver.find_elements(By.CSS_SELECTOR, 'ul#gdasList li div.review_cont')   # reviews에는 리스트가 저장됨.
    print(f"[Page {i+1}] 리뷰 개수:", len(reviews))   # 리뷰가 정상적으로 저장되었는지 확인.

    for review in reviews:
        try:
            user_id = review.find_element(By.CLASS_NAME, 'id').text   # 사용자 id
            skin_type = review.find_element(By.CLASS_NAME, 'user_info').text   # 피부 타입
            rating = len(review.find_elements(By.CSS_SELECTOR, '.review_rating .point.full'))   # ★★★★☆
            criteria = [e.text for e in review.find_elements(By.CSS_SELECTOR, '.review_evaluation span')]   # 평가요소(발색력, 지속력 등등)
            review_text = review.find_element(By.CLASS_NAME, 'txt_inner').text   # 리뷰 본문

            data.append({
                'ID': user_id,
                'Skin_Type': skin_type,
                'Rating': rating,
                'Criteria': ','.join(criteria),
                'Review_Text': review_text
            })
        except Exception as e:
            print("Error: ", e)
            continue
        
    # 다음 페이지 클릭
    try:
        next_btn = driver.find_element(By.CSS_SELECTOR, '.pageing a.next')
        if 'disabled' in next_btn.get_attribute('class'):
            print("📄 마지막 페이지 도달")
            break
        next_btn.click()
        time.sleep(2)
    except Exception as e:
        print("⛔ 다음 페이지 버튼 클릭 실패:", e)
        break

❌ 리뷰 탭 또는 리뷰 목록 로딩 실패
[Page 1] 리뷰 개수: 0
⛔ 다음 페이지 버튼 클릭 실패: Message: element not interactable
  (Session info: chrome=134.0.6998.178)
Stacktrace:
	GetHandleVerifier [0x0090C7F3+24435]
	(No symbol) [0x00892074]
	(No symbol) [0x0076055E]
	(No symbol) [0x007A9C6B]
	(No symbol) [0x0079E685]
	(No symbol) [0x007CD7BC]
	(No symbol) [0x0079E114]
	(No symbol) [0x007CDA34]
	(No symbol) [0x007EF20A]
	(No symbol) [0x007CD5B6]
	(No symbol) [0x0079C54F]
	(No symbol) [0x0079D894]
	GetHandleVerifier [0x00C170A3+3213347]
	GetHandleVerifier [0x00C2B0C9+3295305]
	GetHandleVerifier [0x00C2558C+3271948]
	GetHandleVerifier [0x009A7360+658144]
	(No symbol) [0x0089B27D]
	(No symbol) [0x00898208]
	(No symbol) [0x008983A9]
	(No symbol) [0x0088AAC0]
	BaseThreadInitThunk [0x76665D49+25]
	RtlInitializeExceptionChain [0x76F9CE3B+107]
	RtlGetAppContainerNamedObjectPath [0x76F9CDC1+561]



In [ ]:
# 6. 저장
df = pd.DataFrame(data)
df.to_csv("oliveyoung_{prodct_id}_reviews.csv", index=False, encoding='utf-8-sig')

driver.quit()

[코드 설명]
- #3에서 인덱스 번호를 바꿔가면서 csv파일을 저장할 경우 기존 파일에 새로운 데이터들이 덮어씌워지는 문제 발생.
- 따라서 제품명 별로 구별하여 csv파일 생성.

또 다른 방법: csv저장 모드를 `append` 모드로 바꾸기
``` python
df.to_csv("oliveyoung_reviews.csv", index=False, encoding='utf-8=sig', mode='a', header=False)
```
- 이 경우 첫 번째 제품을 수집할 때만 `header=True`로 해야 CSV에 컬럼명이 들어감.
- 이후 제품들에는 `header=False`로 해야 컬럼명이 중복 저장되지 않음.